In [1]:
import pandas as pd

In [7]:
enru = pd.read_csv("./lookup results/yandex_lookup_en_ru_lemmas_only_WHOLE_LIST.csv")

In [8]:
enru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 5 columns):
Unnamed: 0        1410 non-null int64
word              1410 non-null object
local_word        1410 non-null object
examples          1410 non-null object
local_examples    1410 non-null object
dtypes: int64(1), object(4)
memory usage: 55.2+ KB


In [11]:
enru.head(10)

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,для,"['for the bank', 'for last month', 'for one ye...","['для банка', 'за последний месяц', 'на один г..."
1,1,for,ради,['for god'],['ради бога']
2,2,for,в течение,[],[]
3,3,for,в отношении,[],[]
4,4,for,для целей,[],[]
5,5,for,ибо,[],[]
6,6,for,чтобы,[],[]
7,7,before,до,['before the war'],['до войны']
8,8,before,впереди,[],[]
9,9,before,пред,[],[]


In [13]:
enrufr_naive = pd.read_csv("./lookup results/naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")
enrufr_naive.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe..."
1,1,for,pour,['for god'],['pour dieu']
2,2,for,au cours de,[],[]
3,3,for,à l'égard de,[],[]
4,4,for,à des fins,[],[]


In [16]:
enrufr_naive['russian_word'] = list(enru['local_word'])

In [19]:
del (enrufr_naive['Unnamed: 0'])
enrufr_naive.head()

,word,local_word,examples,local_examples,russian_word
0,for,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe...",для
1,for,pour,['for god'],['pour dieu'],ради
2,for,au cours de,[],[],в течение
3,for,à l'égard de,[],[],в отношении
4,for,à des fins,[],[],для целей


In [21]:
column = ['word','russian_word', 'local_word','examples','local_examples']
enrufr_naive = enrufr_naive.reindex(columns=column)

In [26]:
enrufr_naive.head(60)

,word,russian_word,local_word,examples,local_examples
0,for,для,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe..."
1,for,ради,pour,['for god'],['pour dieu']
2,for,в течение,au cours de,[],[]
3,for,в отношении,à l'égard de,[],[]
4,for,для целей,à des fins,[],[]
5,for,ибо,car,[],[]
6,for,чтобы,pour,[],[]
7,before,до,avant,['before the war'],['avant la guerre']
8,before,впереди,en avant de,[],[]
9,before,пред,devant,[],[]


In [24]:
enrufr_naive.to_csv("enrufr_naive_transation_yandex_lookup_whole_list.csv")

# match systran eng fr

In [106]:
systran_en_fr = pd.read_csv("systran_en_fr.csv")
systran_en_fr.head()

,Unnamed: 0,word,local_word,examples,local_examples,examples_and_definitions
0,0,word,mot,key word|last word|signal word|first words|,mot clé | dernier mot | mot indicateur | premi...,key word|last word|signal word|first words|ter...
1,1,word,parole,fine words|word of god|kind word|word of the l...,belle parole | parole de dieu | aimables parol...,fine words|word of god|kind word|word of the l...
2,2,word,terme,following words|search word|operative word|eng...,termes suivants | terme de recherche | terme p...,following words|search word|operative word|eng...
3,3,word,propos,written word|wise words|clear words|warm words|,propos écrits ici | propos sages | propos clai...,written word|wise words|clear words|warm words...
4,4,word,mention,words of limitation|,mention de limitation |,words of limitation|particular | reference | m...


In [107]:
systran_en_ru = pd.read_csv("systran_en_ru.csv")
systran_en_ru.head(35)

,Unnamed: 0,word,local_word,examples,local_examples,examples_and_definitions
0,0,word,слово,key word|english word|first word|good word|,ключевое слово | английское слово | первое сло...,key word|english word|first word|good word|spe...
1,1,and,и,empty_cell,empty_cell,empty_cell
2,2,and,а,empty_cell,empty_cell,but | while |
3,3,and,и,He thinks of you day and night|,Он думает о вас день и ночь |,He thinks of you day and night|
4,4,and,но,It's an embarrassing joke and not an insult|,"Это неуместная шутка, а не оскорбление |",It's an embarrassing joke and not an insult|
5,5,and,а,This is a squid and not an octopus|,"Это кальмар, а не осьминог |",This is a squid and not an octopus|
6,6,that,так,empty_cell,empty_cell,as | so | as well as | how | so that | both |
7,7,that,то,empty_cell,empty_cell,empty_cell
8,8,that,который / кто / тот который,I know the guy that sold you the disk|,"Я знаю парня, который продал тебе диск |",I know the guy that sold you the disk|
9,9,that,тот,On that evening I met Natalie|Do you see those...,В тот вечер я встретил Натали | Видишь те две ...,On that evening I met Natalie|Do you see those...


In [24]:
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_distances
from  sklearn.metrics.pairwise import cosine_similarity

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import math
import numpy as np

In [3]:
model = api.load('glove-twitter-100')

In [134]:
l1 = "hello my "
l2 = "hello my girlfriend"

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def get_nonstopwords_vectors_sum(lemm_list):
    orig_vector = np.zeros(shape = 100)
    for lemma in lemm_list:
        if lemma not in stopWords:
            #print (lemma)
            try:
                orig_vector += model[lemma]
            except:
                pass
    return orig_vector


def get_lemmatized_line(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return lemmatized_line_list

get_lemmatized_line(l1)

def find_lines_similarity(line_1, line_2):
    lemmatized_l1_list = get_lemmatized_line(line_1)
    lemmatized_l2_list = get_lemmatized_line(line_2)
    #print(lemmatized_l1_list,lemmatized_l2_list )
    l1_vect = get_nonstopwords_vectors_sum(lemmatized_l1_list)
    l2_vect = get_nonstopwords_vectors_sum(lemmatized_l2_list)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2)    
    

0.8311195825025534

In [41]:
cosine_similarity(model['mother'].reshape(1, -1), model['woman'].reshape(1, -1))[0][0]

0.7815035

In [15]:
a = Nan
type(a)

NameError: name 'Nan' is not defined

In [109]:
def update_enf_word(dct_word, senseword, exmpl):
    #print(senseword)
    if senseword in dct_word:
        if exmpl == "empty_cell":
            #print("EMPTTY")
            pass
        else:
            dct_word[senseword]['examples'] += exmpl
    else:
        dct_word[senseword] = {}
        if exmpl == "empty_cell":
            #print("EMPTTY")
            dct_word[senseword]['examples'] = ''
        else:
            dct_word[senseword]['examples'] = exmpl

def csv_to_structure_dict(df):
    new_dict = {}
    for df_ind in range(len(df)):
        df_element = list(df.iloc[df_ind])
        eng_word = df_element[1]
        sense_word = df_element[2]
        examples_context = df_element[5]
        #print(df_element)
        if eng_word not in new_dict:
            new_dict[eng_word] = {}
            update_enf_word(new_dict[eng_word],sense_word, examples_context)
        else:
            update_enf_word(new_dict[eng_word],sense_word, examples_context)
    return new_dict
dct = csv_to_structure_dict(systran_en_ru)

In [110]:
dct['dinner']

{'ужин': {'examples': 'romantic dinner|gala dinner|family dinner|themed dinner|supper | meal | '},
 'обед': {'examples': 'business dinner|excellent dinner|lunch | meal | afternoon | lunchtime | '}}

In [111]:
dct_en_fr = csv_to_structure_dict(systran_en_fr)

In [112]:
dct_en_fr['dinner']

{'dîner': {'examples': 'gala dinner|romantic dinner|mess dinner|business dinners|diner | '},
 'soir': {'examples': 'evening | night | '},
 'souper': {'examples': 'formal dinner|group dinner|supper | diner | '},
 'repas': {'examples': 'theme dinners|medieval dinners|nice dinner|family dinner|meal | food | lunch | '}}

In [137]:
c1 = 'gala dinner|romantic dinner|mess dinner|business dinners|'
c2 = 'romantic dinner|gala dinner|family dinner|themed dinner|'
def get_context_similarity(ctx1, ctx2):
    simialrity_list = []
    #print("ctxs","\n", ctx1,"\n",ctx2 )
    #if "empty_cell" not in ctx1 and "empty_cell" not in ctx2:
    if len(ctx1) != 0 and len(ctx2) != 0:
        for ctx1_el in ctx1.split("|"):
            if ctx1_el.strip() != '':
                #print(ctx1_el)
                for ctx1_e2 in ctx2.split("|"):
                    if ctx1_e2.strip() != '': 
                        #print (ctx1_e2.strip())
                        sim = find_lines_similarity(ctx1_el, ctx1_e2) 
                        #print('SIM',ctx1_el, ctx1_e2,sim)
                        simialrity_list.append(sim)
        #print(simialrity_list)
        return np.median(simialrity_list)
    else:
        return "no_ctx"
    """
    elif "empty_cell" in ctx1 and "empty_cell" in ctx2:
        return "no_ctx_both"
    elif "empty_cell" in ctx1:
        return "no_ctx_l1"
    elif "empty_cell" in ctx2:
        return "no_ctx_l2"   
    """
get_context_similarity(c1, c2)        

0.815088345356733

In [ ]:
"""
def average_similarity(ex_sim, ctx_sim,simil_dict, local_word, compared_word):
    print(local_word, compared_word)
    simil_dict[local_word][compared_word] = 'no_data_from_compared_word'
    if ex_sim == "no_ctx" and ctx_sim == "no_ctx":
        print("here==")
        simil_dict[local_word][compared_word] = "no_context_for_merge"
    elif ex_sim != "no_ctx" and ctx_sim == "no_ctx":
        simil_dict[local_word][compared_word] = ex_sim
        print("here!=",ex_sim)
    elif ex_sim == "no_ctx" and ctx_sim != "no_ctx":
        simil_dict[local_word][compared_word] = ctx_sim
        print("here=!",ctx_sim)
    else:
        simil_dict[local_word][compared_word] = (ex_sim + ctx_sim)/2
        print("here!!",ex_sim, ctx_sim, (ex_sim + ctx_sim)/2)
"""

In [175]:
def get_context_and_examples_sim(senseword_dict_l1, senseword_dict_l2, simil_dict_1,simil_dict_2, 
                                 loc_w_1, loc_w_2, print_output = True):
    if print_output: print(loc_w_1, senseword_dict_l1,"\n", loc_w_2, senseword_dict_l2)
    similarity = get_context_similarity(senseword_dict_l1['examples'], senseword_dict_l2['examples'])
    if print_output: print(loc_w_1, loc_w_2, "examples_sim", similarity,"\n")
        
    if similarity == "no_ctx":
        simil_dict_1[loc_w_1][loc_w_2] = 0
        simil_dict_2[loc_w_2][loc_w_1] = 0
    else:
        simil_dict_1[loc_w_1][loc_w_2] = similarity
        simil_dict_2[loc_w_2][loc_w_1] = similarity

        
def merge_two_lang_structures(lang_dict_1, lang_dict_2):
    sim_dict_l1 = {}
    sim_dict_l2 = {}
    for local_1_senseword in lang_dict_1.keys():
        sim_dict_l1[local_1_senseword] = {}
        for local_2_senseword in lang_dict_2.keys():
            if local_2_senseword not in sim_dict_l2:
                sim_dict_l2[local_2_senseword] = {}
            sim = get_context_and_examples_sim(lang_dict_1[local_1_senseword], lang_dict_2[local_2_senseword],
                                              sim_dict_l1,sim_dict_l2, local_1_senseword, local_2_senseword)
            
            #break
        #break
    print("INNER_MERGER_DICT")
    print(sim_dict_l1)
    print()
    print(sim_dict_l2)
    print("=======\n")
#merge_two_lang_structures(dct['dinner'], dct_en_fr['dinner'])   

def merge_two_dct(big_dict_l1, big_dict_l2):
    count = 0
    global_merged_dict= {}
    for eng_word in big_dict_l1.keys():
        #merge_two_lang_structures(big_dict_l1[eng_word], big_dict_l2[eng_word])
        try:
            merge_two_lang_structures(big_dict_l1[eng_word], big_dict_l2[eng_word])
        except:
            print(eng_word, 'MERGE_PROBLEM')
        count += 1
        if count > 30: break
        
merge_two_dct(dct,dct_en_fr)        


слово {'examples': 'key word|english word|first word|good word|speech | expression | term | '} 
 mot {'examples': 'key word|last word|signal word|first words|term | remark | '}
слово mot examples_sim 0.6725331071042714 

слово {'examples': 'key word|english word|first word|good word|speech | expression | term | '} 
 parole {'examples': 'fine words|word of god|kind word|word of the lord|floor | speech | lyric | hon | voice | '}
слово parole examples_sim 0.562559223050436 

слово {'examples': 'key word|english word|first word|good word|speech | expression | term | '} 
 terme {'examples': 'following words|search word|operative word|english words|term | end | stop | '}
слово terme examples_sim 0.644972097261103 

слово {'examples': 'key word|english word|first word|good word|speech | expression | term | '} 
 propos {'examples': 'written word|wise words|clear words|warm words|comment | remark | hon | rhetoric | pertinence | suitableness | rightness | aptness | '}
слово propos examples_sim 0

в car examples_sim 0.0 

в {'examples': 'in | to | at | on | into | '} 
 pour {'examples': 'to | on | '}
в pour examples_sim 0.0 

в {'examples': 'in | to | at | on | into | '} 
 de {'examples': 'of | in | from | to | on | by | '}
в de examples_sim 0.0 

в {'examples': 'in | to | at | on | into | '} 
 à {'examples': 'to | in | at | on | with | '}
в à examples_sim 0.0 

в {'examples': 'in | to | at | on | into | '} 
 dans {'examples': 'in | into | on | within | to | '}
в dans examples_sim 0.0 

по {'examples': 'on | by | at | according to | '} 
 car {'examples': 'because | as | since | '}
по car examples_sim 0.0 

по {'examples': 'on | by | at | according to | '} 
 pour {'examples': 'to | on | '}
по pour examples_sim 0.0 

по {'examples': 'on | by | at | according to | '} 
 de {'examples': 'of | in | from | to | on | by | '}
по de examples_sim 0.0 

по {'examples': 'on | by | at | according to | '} 
 à {'examples': 'to | in | at | on | with | '}
по à examples_sim 0.0 

по {'examples': '

этот ce examples_sim 0.2259541362860551 

этот {'examples': 'Do you recognise this object?|'} 
 ce... -ci / celui-ci {'examples': "Which one should I buy ? This one or that one ?|This computer is the one I want|This time, I won't forget you|"}
этот ce... -ci / celui-ci examples_sim 0.36605328648269636 

этот {'examples': 'Do you recognise this object?|'} 
 ce / ceci {'examples': 'This is what I heard|These are my friends|This is my home|'}
этот ce / ceci examples_sim 0.2988971098406829 

это {'examples': 'I will explain this to you more fully later|'} 
 ce {'examples': 'this armchair|this story|this man|this friend|these seats|'}
это ce examples_sim 0.5897737081761073 

это {'examples': 'I will explain this to you more fully later|'} 
 ce... -ci / celui-ci {'examples': "Which one should I buy ? This one or that one ?|This computer is the one I want|This time, I won't forget you|"}
это ce... -ci / celui-ci examples_sim 0.772641587255732 

это {'examples': 'I will explain this to you mor

быть être examples_sim 0.4252324608045508 

быть {'examples': 'to have time|to have their hands full|to have a product|to have nuclear weapons|be | include | feature | get | come | go | offer | become | '} 
 posséder {'examples': 'to have legal personality|to have a balcony|to have a car|to have a beach|possess | offer | feature | own | hold | boast | '}
быть posséder examples_sim 0.3157288641725179 

быть {'examples': 'to have time|to have their hands full|to have a product|to have nuclear weapons|be | include | feature | get | come | go | offer | become | '} 
 faire {'examples': 'to have a break|to have a quick dip|to have a party|to have confidence in the Commissioner|do | make | do so | make | take | do | give | get | '}
быть faire examples_sim 0.4404885889080683 

быть {'examples': 'to have time|to have their hands full|to have a product|to have nuclear weapons|be | include | feature | get | come | go | offer | become | '} 
 prendre {'examples': 'to have a drink|to have breakfast|

обладать prendre examples_sim 0.04690289979412011 

обладать {'examples': 'to have legitimacy|to have its own local particularities|possess | wield | own | possess | '} 
 obtenir {'examples': 'to have nicotine|to have a mile-long list|to have a copy|to have a valid visa|obtain | get | achieve | receive | obtain | see | secure | seek | '}
обладать obtenir examples_sim 0.16633613829750657 

обладать {'examples': 'to have legitimacy|to have its own local particularities|possess | wield | own | possess | '} 
 présenter {'examples': 'to have the following characteristics|to have tremor|to have individual character|to have the following minimum characteristics|present | submit | provide | introduce | make | table | show | put forward | file | '}
обладать présenter examples_sim 0.16119791062497837 

обладать {'examples': 'to have legitimacy|to have its own local particularities|possess | wield | own | possess | '} 
 bénéficier {'examples': 'to have any rights|to have that protection|to have y

...

выходить sortir examples_sim 0.3463172002113541 

выходить {'examples': 'to go out into the streets|go | come out | face | extend | enter | overlook | emerge | withdraw | '} 
 s'éteindre {'examples': 'off-~-turn off | extinguish | down-~-power down | shut down | go off | lose power | shut off | away-~-die away | '}
выходить s'éteindre examples_sim 0.20995837878181622 

выходить {'examples': 'to go out into the streets|go | come out | face | extend | enter | overlook | emerge | withdraw | '} 
 se retirer {'examples': 'to go out of business|withdraw | retire | pull out | retreat | out-~-opt out | get out | step-down | resign | '}
выходить se retirer examples_sim 0.2904166581653816 

пойти {'examples': 'go | walk | go | '} 
 sortir {'examples': 'to go out at night|to go out tonight|to go out last night|to go out into the night alone|leave | get out | release | emerge | out-~-come out | come out of | exit | leave | '}
пойти sortir examples_sim 0.7559996315006663 

пойти {'examples': 'go | 

секретный secret examples_sim 0.3645916523023321 

секретный {'examples': 'confidential | sensitive | covert | secretive | undercover | '} 
 confidentiel {'examples': 'confidential | sensitive | '}
секретный confidentiel examples_sim 0.40017749068832176 

секретный {'examples': 'confidential | sensitive | covert | secretive | undercover | '} 
 passe {'examples': 'chap secret|mutual chap secret|password | pass | '}
секретный passe examples_sim 0.2427130083031789 

секретный {'examples': 'confidential | sensitive | covert | secretive | undercover | '} 
 mystère {'examples': 'mystery | secrecy | puzzle | '}
секретный mystère examples_sim 0.28375488918399555 

тайный {'examples': 'clandestine | covert | privy | arcane | insidious | esoteric | '} 
 secret {'examples': 'so secret|still secret|highly secret|secretive | hidden | confidential | covert | trade secret|little secret|shared secret|business secrets|secrecy | confidentiality | '}
тайный secret examples_sim 0.208055934114401 

тайный 

настольный soumettre examples_sim 0.28646320376080764 

стол {'examples': 'round table|dining table|pool table|negotiating table|desk | desktop | '} 
 tableau {'examples': 'following table|nutrition facts table|summary table|financial table|array | painting | chart | picture | '}
стол tableau examples_sim 0.4892176309232753 

стол {'examples': 'round table|dining table|pool table|negotiating table|desk | desktop | '} 
 table {'examples': 'round table|coffee table|database table|data tables|payola | '}
стол table examples_sim 0.6760474073173695 

стол {'examples': 'round table|dining table|pool table|negotiating table|desk | desktop | '} 
 déposer {'examples': 'to table amendments|to table a document|to table this document|tabled in the house|file | deposit | submit | lodge | place | introduce | present | file | '}
стол déposer examples_sim 0.4593532972976105 

стол {'examples': 'round table|dining table|pool table|negotiating table|desk | desktop | '} 
 présenter {'examples': 'to table

создать établir examples_sim 0.5447644371510256 

создать {'examples': 'to create a vacuum|to create conditions|to create a system|to create institutions|establish | make | build | setup | form | found | develop | '} 
 engendrer {'examples': 'to even create competition|to create poverty|to create illness|to create a legal presumption|cause | lead | generate | result | give rise to | engender | incur | breed | '}
создать engendrer examples_sim 0.4910333841751753 

создавать {'examples': 'to create a world|to create problems|to create the impression|to create the circumstances|make | build | produce | establish | generate | form | setup | '} 
 créer {'examples': 'to create jobs|to create an environment|to create a situation|to create applications|establish | setup | develop | build | found | '}
создавать créer examples_sim 0.583005678133397 

создавать {'examples': 'to create a world|to create problems|to create the impression|to create the circumstances|make | build | produce | establis

уютный chauffer examples_sim 0.21108157102819777 

горячий {'examples': 'hot | heated | ardent | '} 
 chaleureux {'examples': 'particularly warm|extremely warm|quite warm|friendly | cosy | cordial | '}
горячий chaleureux examples_sim 0.34284707665010516 

горячий {'examples': 'hot | heated | ardent | '} 
 chaud {'examples': 'still warm|relatively warm|warm enough|so warm|hot | cooked | '}
горячий chaud examples_sim 0.4410839467249028 

горячий {'examples': 'hot | heated | ardent | '} 
 tiède {'examples': 'lukewarm | tepid | half-hearted | tempered | '}
горячий tiède examples_sim 0.14622675208483193 

горячий {'examples': 'hot | heated | ardent | '} 
 convivial {'examples': 'friendly | user-friendly | convivial | easy-to-use | cosy | easy to use | '}
горячий convivial examples_sim 0.21498966217627136 

горячий {'examples': 'hot | heated | ardent | '} 
 accueillant {'examples': 'friendly | cosy | hospitable | homely | homey | appealing | '}
горячий accueillant examples_sim 0.214599058590

In [191]:
def merge_dicts(l1_l2_dict,l2_to_l1, l1_to_l2):
    for local_word_1 in l1_l2_dict.keys():
        sorted_sim = sorted(l1_l2_dict[local_word_1].items(), key=lambda kv: kv[1], reverse = True)
        if sorted_sim[0][1] <= 0.6:
            print("no suitable connections for ", local_word_1)
            l1_to_l2[local_word_1] = "no_equality"
        else:
            if local_word_1 in l1_to_l2:
                if sorted_sim[0][0] not in l1_to_l2[local_word_1]:
                    l1_to_l2[local_word_1].append(sorted_sim[0][0])
            else:
                l1_to_l2[local_word_1] = []
                l1_to_l2[local_word_1].append(sorted_sim[0][0])
            print(local_word_1, "sorted_sim[0][1]",sorted_sim[0][1])
            
            if sorted_sim[0][0] in l2_to_l1:  
                if local_word_1 not in l2_to_l1[sorted_sim[0][0]]:
                    l2_to_l1[sorted_sim[0][0]].append(local_word_1)
            else:
                l2_to_l1[sorted_sim[0][0]] = []
                l2_to_l1[sorted_sim[0][0]].append(local_word_1)
                
ru_fr_dict = {'обойти': {'se promener': 0.16193252812028638, 'se balader': 0.15641367945030674, "s'éloigner": 0.162819447439424, 'promener': 0.16490256085461272}, 'вращаться': {'se promener': 0.4123024692238632, 'se balader': 0.36482488114980133, "s'éloigner": 0.19768349945429559, 'promener': 0.22889580396221318}, 'ездить по миру': {'se promener': 0, 'se balader': 0, "s'éloigner": 0, 'promener': 0}, 'часто путешествовать': {'se promener': 0, 'se balader': 0, "s'éloigner": 0, 'promener': 0}, 'хватать': {'se promener': 0.36273753561810446, 'se balader': 0.2975160600112141, "s'éloigner": 0.3047695342194968, 'promener': 0.29944649457673916}, 'пролетать': {'se promener': 0.6350658803284455, 'se balader': 0.3733952107024293, "s'éloigner": 0.4256126771370779, 'promener': 0.6414917193832451}, 'ходить вокруг': {'se promener': 0, 'se balader': 0, "s'éloigner": 0, 'promener': 0}}
fr_ru_dict = {'se promener': {'обойти': 0.16193252812028638, 'вращаться': 0.4123024692238632, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.36273753561810446, 'пролетать': 0.6350658803284455, 'ходить вокруг': 0}, 'se balader': {'обойти': 0.15641367945030674, 'вращаться': 0.36482488114980133, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.2975160600112141, 'пролетать': 0.3733952107024293, 'ходить вокруг': 0}, "s'éloigner": {'обойти': 0.162819447439424, 'вращаться': 0.19768349945429559, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.3047695342194968, 'пролетать': 0.4256126771370779, 'ходить вокруг': 0}, 'promener': {'обойти': 0.16490256085461272, 'вращаться': 0.22889580396221318, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.29944649457673916, 'пролетать': 0.6414917193832451, 'ходить вокруг': 0}}
def create_matching_structure(l1_l2_dict, l2_l1_dict, l1_to_l2, l2_to_l1):
    print("FIRST LANG TO SECOND")
    merge_dicts(l1_l2_dict, l1_to_l2, l2_to_l1)
    """
    for local_word_1 in l1_l2_dict.keys():
        sorted_sim = sorted(l1_l2_dict[local_word_1].items(), key=lambda kv: kv[1], reverse = True)
        print(local_word_1,)
        if sorted_sim[0][1] == "no_context_for_merge":
            l1_to_l2[local_word_1] = "no_equality"
        elif sorted_sim[0][1] <= 0.6:
            l1_to_l2[local_word_1] = "no_equality"
        else:
            l1_to_l2[local_word_1] = sorted_sim[0][0]
            #print(local_word_1, "sorted_sim[0][1]",sorted_sim[0][1])
            l2_to_l1[sorted_sim[0][0]] = local_word_1
    """
    print('\n',l1_to_l2)
    print()
    print(l2_to_l1)
    print("\nSECOND LANG TO FIRST")
    merge_dicts(l2_l1_dict, l2_to_l1, l1_to_l2)
    print('\n',l1_to_l2)
    print()
    print(l2_to_l1)
    print()
    

l1_to_l2_connect = {}
l2_to_l1_connect = {}

create_matching_structure(ru_fr_dict, fr_ru_dict,l1_to_l2_connect, l2_to_l1_connect)    
    

FIRST LANG TO SECOND
no suitable connections for  обойти
no suitable connections for  вращаться
no suitable connections for  ездить по миру
no suitable connections for  часто путешествовать
no suitable connections for  хватать
пролетать sorted_sim[0][1] 0.6414917193832451
no suitable connections for  ходить вокруг

 {'promener': ['пролетать']}

{'обойти': 'no_equality', 'вращаться': 'no_equality', 'ездить по миру': 'no_equality', 'часто путешествовать': 'no_equality', 'хватать': 'no_equality', 'пролетать': ['promener'], 'ходить вокруг': 'no_equality'}

SECOND LANG TO FIRST
se promener sorted_sim[0][1] 0.6350658803284455
no suitable connections for  se balader
no suitable connections for  s'éloigner
promener sorted_sim[0][1] 0.6414917193832451

 {'promener': ['пролетать'], 'se promener': ['пролетать'], 'se balader': 'no_equality', "s'éloigner": 'no_equality'}

{'обойти': 'no_equality', 'вращаться': 'no_equality', 'ездить по миру': 'no_equality', 'часто путешествовать': 'no_equality', 'х

In [142]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted(x.items(), key=lambda kv: kv[1])

[(0, 0), (2, 1), (1, 2), (4, 3), (3, 4)]